### Problem to solve

The goal of this issue is to collect a minimum of 1000 reviews for any 10 cryptocurrency-related applications (100 each). These could be the apps of the blockchain wallets, crypto custodians, or any crypto projects.

1. Collect reviews for the apps of your choice and make sure that you define where the reviews are coming from - Apple App Store or Google Play Store.
2. Identify sentiment score for each review using one of the existing sentiment analysis tools.
3. Identify either geolocation, or language for each review.
4. For the final deliverable, create:

-  CSV-file with the following structure: `app name`, `username`, `timestamp`, `app review text`, `sentiment score`, `country (language)`, `marketplace`*

*`marketplace` - Apple App Store or Google Play Store

- short report with graphs (based on the reviews that you collected) and basic descriptive statistics

### Installing and Importing

In [ ]:
pip install google_play_scraper

In [ ]:
pip install app_store_scraper

In [8]:
import pandas as pd
import numpy as np
from google_play_scraper import app, Sort, reviews_all, reviews
from  app_store_scraper import AppStore
import json, os, uuid
from tqdm import tqdm


### Defining Variables

In [2]:
apps_names = ["Trust: Crypto & Bitcoin Wallet",  #1
            "Binance: BTC, Crypto and NFTS",     #2
            "Blockchain.com Wallet: Buy BTC",    #3
            "Coinbase: Buy Bitcoin & Ether ",    #4
            "Exodus: Crypto Bitcoin Wallet",     #5
            "EXMO: buy, sell crypto and BTC",    #6
            "Enjin: Crypto & NFT Wallet",        #7
            "HyperPay :Wallet Crypto & Card",    #8
            "Coinomi: Crypto Bitcoin Wallet",    #9
            "Guarda Crypto Bitcoin Wallet"       #10
]
apps_info_g={"Trust: Crypto & Bitcoin Wallet":"com.wallet.crypto.trustapp",
            "Binance: BTC, Crypto and NFTS":"com.binance.dev",
            "Blockchain.com Wallet: Buy BTC":"piuk.blockchain.android",
            "Coinbase: Buy Bitcoin & Ether ":"com.coinbase.android",
            "Exodus: Crypto Bitcoin Wallet":"exodusmovement.exodus",
            "EXMO: buy, sell crypto and BTC":"com.exmo",
            "Enjin: Crypto & NFT Wallet":"com.enjin.mobile.wallet",
            "HyperPay :Wallet Crypto & Card":"com.legendwd.hyperpayW",
            "Coinomi: Crypto Bitcoin Wallet":"com.coinomi.wallet",
            "Guarda Crypto Bitcoin Wallet":"com.crypto.multiwallet"}
apps_info_a={"Trust: Crypto & Bitcoin Wallet":("trust-crypto-bitcoin-wallet","1288339409"),
            "Binance: BTC, Crypto and NFTS":("binance-buy-bitcoin-crypto","1436799971"),
            "Blockchain.com Wallet: Buy BTC":("blockchain-com-wallet-buy-btc","493253309"),
            "Coinbase: Buy Bitcoin & Ether ":("coinbase-buy-bitcoin-ether","886427730"),
            "Exodus: Crypto Bitcoin Wallet":("exodus-crypto-bitcoin-wallet","1414384820"),
            "EXMO: buy, sell crypto and BTC":("exmo-cryptocurrency-exchange","1505496232"),
            "Enjin: Crypto & NFT Wallet":("enjin-nft-crypto-wallet","1349078375"),
            "HyperPay :Wallet Crypto & Card":("hyperpay-crypto-btc-wallet","1354755812"),
            "Coinomi: Crypto Bitcoin Wallet":("coinomi-wallet","1333588809"),
            "Guarda Crypto Bitcoin Wallet":("guarda-crypto-wallet-bitcoin","1442083982")}

In [3]:
n_reviews_app_source = 50 # per app per source 
def_lang='en'
def_country='us'

In [4]:
apps_info_a[apps_names[0]][1] # getting id of an application

'1288339409'

### Languages and Countries

In [47]:
Countries = ['us', 'fr','de','es','bg']
languages = { "us":"en",
     "fr":"fr",
     "de":"de",
     "es":"es",
     "bg":"bg"}
     
'''lang_country = {"United States": ("us","en"),
    "France": ("fr","fr"),
    "Germany": ("de","de"),
    "Spain": ("es","es"),
    "Bulgaria": ("bg","bg"),}'''


### Scraping Functions

In [29]:
def fix_format (app_name, fetched_reviews, source,lang =def_lang, country=def_country):
    '''
    fix_format is a function that reformat the scrapped reviews
    : app_name: the name of the app to scrape reviews for
    : fetched_reviews: list of fetched reviews
    : source: 'a': scrape from Apple store, 'g': scrape from Google Play Store
    : count: number of reviews to be scrapped
    : lang: language of reviews to be scrapped
    : country: country where to scratch reviews from  
    '''
    
    if source == 'g':
        fixed_reviews = [{'app name':app_name,
             'username':review['userName'],
             'timestamp':review['at'],
             'app review text':review['content'],
             'feteched_language': lang,
             'feteched_country': country,
             'marketplace':"Google Play"
             } for review in fetched_reviews]
        return fixed_reviews  
        
    elif source == 'a':
        fixed_reviews = [{'app name':app_name,
             'username':review['userName'],
             'timestamp':review['date'],
             'app review text':review['title']+' '+review['review'],
             'feteched_language': lang,
             'feteched_country': country,
             'marketplace':"App Store"
             }for review in fetched_reviews]
        return fixed_reviews  
    else:
        print(f'Source must be either {0} or {1}','g : Google Play', 'a : Apple Store' )

        

In [16]:
def scrape(app_name, source, count=n_reviews_app_source, lang =def_lang, country=def_country):
    '''
    scrape is a function that uses google_play_ scrapper or app_store_scraper to scrape an app reviews.
    : app_name: the name of the app to scrape reviews for
    : source: 'a': scrape from Apple store, 'g': scrape from Google Play Store
    : count: number of reviews to be scrapped
    : lang: language of reviews to be scrapped
    : country: country where to scratch reviews from    
    '''
    
    if source == 'g':       
        g_reviews, _ = reviews(apps_info_g[app_name],
                lang=lang,  # defaults to 'en'
                country=country,  # defaults to 'us'
                count=count  ) # defaults to 30 (= Google's maximum)
        fixed_reviews = fix_format(app_name, g_reviews,source,lang,country)
        return fixed_reviews

    elif source == 'a':
        a_reviews = AppStore(country,apps_info_a[app_name][0],apps_info_a[app_name][1])
        a_reviews.review(how_many=count)
        fixed_reviews=( fix_format(app_name, a_reviews.reviews,source,lang,country))
        return fixed_reviews
    
    else:
        print(f'Source must be either {0} or {1}','g : Google Play', 'a : Apple Store' )


In [17]:
app_reviews = []
for app in tqdm(apps_names):
    g_reviews = scrape(app, 'g', count=2)
    app_reviews.extend(g_reviews)
    a_reviews = scrape(app, 'a', count=2)
    app_reviews.extend(a_reviews)

  0%|          | 0/10 [00:00<?, ?it/s]2022-09-27 17:34:18,003 [INFO] Base - Initialised: AppStore('us', 'trust-crypto-bitcoin-wallet', 1288339409)
2022-09-27 17:34:18,004 [INFO] Base - Ready to fetch reviews from: https://apps.apple.com/us/app/trust-crypto-bitcoin-wallet/id1288339409
2022-09-27 17:34:18,927 [INFO] Base - [id:1288339409] Fetched 20 reviews (20 fetched in total)
 10%|█         | 1/10 [00:02<00:26,  2.99s/it]2022-09-27 17:34:20,277 [INFO] Base - Initialised: AppStore('us', 'binance-buy-bitcoin-crypto', 1436799971)
2022-09-27 17:34:20,278 [INFO] Base - Ready to fetch reviews from: https://apps.apple.com/us/app/binance-buy-bitcoin-crypto/id1436799971
2022-09-27 17:34:20,784 [INFO] Base - [id:1436799971] Fetched 20 reviews (20 fetched in total)
 20%|██        | 2/10 [00:04<00:18,  2.32s/it]2022-09-27 17:34:22,117 [INFO] Base - Initialised: AppStore('us', 'blockchain-com-wallet-buy-btc', 493253309)
2022-09-27 17:34:22,118 [INFO] Base - Ready to fetch reviews from: https://app

In [19]:
len(app_reviews)

220

### Current Results to DF to CSV

In [23]:
feteched_df = pd.DataFrame(app_reviews)

In [24]:
feteched_df

,app name,username,timestamp,app review text,feteched_language,feteched_country,marketplace
0,Trust: Crypto & Bitcoin Wallet,SureshPatel 0719,2022-09-26 17:22:42,good,en,us,Google Play
1,Trust: Crypto & Bitcoin Wallet,Ravinder Singh,2022-09-26 17:16:17,After update my trust wallet is not open pleas...,en,us,Google Play
2,Trust: Crypto & Bitcoin Wallet,Frank0901,2022-09-18 06:38:11,Awesome App Review I was so impatient to carry...,en,us,App Store
3,Trust: Crypto & Bitcoin Wallet,dontcallmepablo,2021-10-27 14:32:27,Mixed Reviews I’ve been using this app for a f...,en,us,App Store
4,Trust: Crypto & Bitcoin Wallet,"k,nguyen",2021-05-14 09:52:31,Need my money back please. Please read it befo...,en,us,App Store
...,...,...,...,...,...,...,...
215,Guarda Crypto Bitcoin Wallet,Spanky80-8071,2021-08-04 23:52:58,Guarda is top notch Their wallet works for me ...,en,us,App Store
216,Guarda Crypto Bitcoin Wallet,Ryanc016,2021-08-26 06:23:19,Extremely difficult to use and unnecessary fee...,en,us,App Store
217,Guarda Crypto Bitcoin Wallet,dont send money here,2021-01-30 19:08:59,Bad platform... lost half of the money because...,en,us,App Store
218,Guarda Crypto Bitcoin Wallet,h0nk h0nk acr0ss y0ur brain,2021-04-08 15:53:49,"extremely good easy to use, among (sus?) other...",en,us,App Store


### CSV to df

In [27]:
feteched_df.to_csv('current_english_fetched.csv',index=False)  

In [28]:
df = pd.read_csv('current_english_fetched.csv')
df

,app name,username,timestamp,app review text,feteched_language,feteched_country,marketplace
0,Trust: Crypto & Bitcoin Wallet,SureshPatel 0719,2022-09-26 17:22:42,good,en,us,Google Play
1,Trust: Crypto & Bitcoin Wallet,Ravinder Singh,2022-09-26 17:16:17,After update my trust wallet is not open pleas...,en,us,Google Play
2,Trust: Crypto & Bitcoin Wallet,Frank0901,2022-09-18 06:38:11,Awesome App Review I was so impatient to carry...,en,us,App Store
3,Trust: Crypto & Bitcoin Wallet,dontcallmepablo,2021-10-27 14:32:27,Mixed Reviews I’ve been using this app for a f...,en,us,App Store
4,Trust: Crypto & Bitcoin Wallet,"k,nguyen",2021-05-14 09:52:31,Need my money back please. Please read it befo...,en,us,App Store
...,...,...,...,...,...,...,...
215,Guarda Crypto Bitcoin Wallet,Spanky80-8071,2021-08-04 23:52:58,Guarda is top notch Their wallet works for me ...,en,us,App Store
216,Guarda Crypto Bitcoin Wallet,Ryanc016,2021-08-26 06:23:19,Extremely difficult to use and unnecessary fee...,en,us,App Store
217,Guarda Crypto Bitcoin Wallet,dont send money here,2021-01-30 19:08:59,Bad platform... lost half of the money because...,en,us,App Store
218,Guarda Crypto Bitcoin Wallet,h0nk h0nk acr0ss y0ur brain,2021-04-08 15:53:49,"extremely good easy to use, among (sus?) other...",en,us,App Store


#### next
- sentiment analysis
- scrape regradless of the language (beautifulsoup ?)
- sentiment analysis regradless of the language
- describtive statics
- report

### Detect language of sentence

### Sentiment analysis
no matter what the language is